# Transformer Model

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# https://huggingface.co/docs/transformers/training

In [2]:
df = pd.read_csv('untitled.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
df['IsBodyShopRepair'].value_counts()

IsBodyShopRepair
N    614
Y     72
Name: count, dtype: int64

In [101]:
df.shape

(686, 2)

In [103]:
df

,Damage,IsBodyShopRepair
6,"FRNT BMPR, ALGNMT, HOOD ...",N
7,"R/FR BUMPER,SCR R/FENDER ...",N
8,REAR BUMPER SCRAPED ...,N
9,SCUFF-DVR FT BMPR/QTR PNL ...,N
10,"REAR END,BUMPER,QRT ...",N
...,...,...
968,NONE,N
982,license PLATE IS EXPIRED. THERE IS SOMETHING W...,N
985,NONE-JUMP START ASSISTANCE \r\n\r\nPOSSIBLE TO...,N
991,unknown,N


In [104]:
import string
def clean(x):
    # x = x.tolist()[0]
    x = x.lower()
    new_str = ''
    for char in x:
        if char not in string.punctuation:
            new_str = new_str + char
    return new_str.strip()

In [105]:
df['text'] = df['Damage'].apply(lambda x: clean(x))

In [106]:
df.rename(columns={'IsBodyShopRepair':'label'},inplace=True)

In [107]:
df.drop('Damage',axis=1,inplace=True)

In [109]:
df['label'] = df['label'].replace(['Y','N'],[1,0])

In [110]:
df

,label,text
6,0,frnt bmpr algnmt hood
7,0,rfr bumperscr rfender
8,0,rear bumper scraped
9,0,scuffdvr ft bmprqtr pnl
10,0,rear endbumperqrt
...,...,...
968,0,none
982,0,license plate is expired there is something wr...
985,0,nonejump start assistance \r\n\r\npossible tow...
991,0,unknown


In [111]:
df['label'].unique()

array([0, 1], dtype=int64)

In [32]:
!pip install -q datasets

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\avitr\\anaconda3\\lib\\site-packages\\PyYAML-5.4.1.dist-info\\METADATA'



In [112]:
df

,label,text
6,0,frnt bmpr algnmt hood
7,0,rfr bumperscr rfender
8,0,rear bumper scraped
9,0,scuffdvr ft bmprqtr pnl
10,0,rear endbumperqrt
...,...,...
968,0,none
982,0,license plate is expired there is something wr...
985,0,nonejump start assistance \r\n\r\npossible tow...
991,0,unknown


In [113]:
from sklearn.model_selection import train_test_split

In [114]:
df_train,df_val = train_test_split(df,test_size=0.1,random_state=42)

In [115]:
df_train.shape

(617, 2)

In [116]:
df_val.shape

(69, 2)

In [117]:
import datasets
import os
import numpy as np


# prepare dataset for finetuning
train_dataset = datasets.Dataset.from_dict(df_train)
val_dataset = datasets.Dataset.from_dict(df_val)


my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"val":val_dataset})

In [118]:
my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 617
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 69
    })
})

In [119]:
!pip install -q transformers

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\avitr\\anaconda3\\lib\\site-packages\\PyYAML-5.4.1.dist-info\\METADATA'



In [120]:
from transformers import AutoTokenizer

In [121]:
# load Tokenizer from bert-base-uncased checkpoint
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

loading configuration file config.json from cache at C:\Users\avitr/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\avitr/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\vocab.t

In [122]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [123]:
# get tokenized dataset
def tokenize_function(examples):
    # max token length max_length=20, you can change it to max 512
    return tokenizer(examples["text"], padding="max_length", max_length=20, truncation=True)
tokenized_datasets = my_dataset_dict.map(tokenize_function, batched=True)

Map:   0%|          | 0/617 [00:00<?, ? examples/s]

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

In [124]:
my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 617
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 69
    })
})

In [125]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 617
    })
    val: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 69
    })
})

In [126]:
df['label'].unique().tolist()

[0, 1]

In [127]:
class_names = df['label'].unique().tolist()

In [128]:
from transformers import AutoModelForSequenceClassification

# load classifier from bert-base-uncased
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(class_names))

loading configuration file config.json from cache at C:\Users\avitr/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\avitr/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64

In [129]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
!pip install -q evaluate

In [130]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [131]:
# function to compute performance while finetuning
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [132]:
from transformers import TrainingArguments, Trainer
# prepare training Arguments
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                  num_train_epochs = 2, # change here
                                  overwrite_output_dir=True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [133]:
# Trainer for finetuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_metrics,
    # callbacks=[checkpoint_callback]
)

In [134]:
# train the model for finetuning
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 617
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 156


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.176891,0.956522
2,No log,0.267106,0.956522


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=156, training_loss=0.3423589559701773, metrics={'train_runtime': 312.1459, 'train_samples_per_second': 3.953, 'train_steps_per_second': 0.5, 'total_flos': 12682775090400.0, 'train_loss': 0.3423589559701773, 'epoch': 2.0})

In [135]:
# save the model
trainer.save_model("MyOwn_finetuned_model")

Saving model checkpoint to MyOwn_finetuned_model
Configuration saved in MyOwn_finetuned_model\config.json
Model weights saved in MyOwn_finetuned_model\pytorch_model.bin


# Now use Our finetuned Model

In [139]:
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)

# load fientuned model
model_tuned = AutoModelForSequenceClassification.from_pretrained("MyOwn_finetuned_model")

loading configuration file MyOwn_finetuned_model\config.json
Model config BertConfig {
  "_name_or_path": "MyOwn_finetuned_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file MyOwn_finetuned_model\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

In [140]:
model_tuned

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [197]:
from scipy.special import softmax
def inference(text):
    batch = tokenizer.encode(text, return_tensors='pt')
    output = model_tuned(batch)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    labels = [0,1]
    return labels[scores.argmax()]

In [201]:
# prediction for first 50 lines
df['text'][:50].apply(lambda x:inference(x))

6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
Name: text, dtype: int64